In [ ]:
!pip install tfx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 98.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.6/206.6 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv
import os
import tempfile,urllib,zipfile
from tensorflow.python.lib.io import file_io
from tensorflow_data_validation.utils import slicing_util
from tensorflow_metadata.proto.v0.statistics_pb2 import DatasetFeatureStatisticsList, DatasetFeatureStatistics

# Load the dataset

In [ ]:
df = pd.read_csv("/content/diabetic_data.csv",header = 0,na_values = '?')
df.head()

<ipython-input-19-e28750f86a24>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/diabetic_data.csv",header = 0,na_values = '?')


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


# Data splits
## 70% training set
## 15% evaluation set
## 15% serving set

In [ ]:
def prepare_data_splits_from_df(df):
  # 70% of data for generating the training set
  train_len = int(len(df) * 0.7)
  # Remaining 30% of data for evaluation and serving set
  eval_serv_len = len(df) - train_len
  # Evaluation set 15%
  eval_len = eval_serv_len//2
  # Serving set 15%
  serv_len = eval_serv_len - eval_len

  # Split the dataframe into three subsets
  train_df = df.iloc[ : train_len].reset_index(drop=True)
  eval_df = df.iloc[train_len : train_len + eval_len].reset_index(drop = True)
  serv_df = df.iloc[train_len + eval_len : train_len+eval_len+serv_len].reset_index(drop = True)

  # Serving data should not have labels in the dataset
  serv_df = serv_df.drop(['readmitted'],axis = 1)

  return train_df,eval_df,serv_df


# Split the dataset
train_df,eval_df,serv_df = prepare_data_splits_from_df(df)

print(f"Training dataset has {len(train_df)} records")
print(f"Evaluation dataset has {len(eval_df)} records")
print(f"Serving dataset has {len(serv_df)} records")

Training dataset has 71236 records
Evaluation dataset has 15265 records
Serving dataset has 15265 records


# Generate and Visualize Training Data Statistics
## exploratory data analysis should be performed on the training dataset

In [ ]:
# Removing irrelevant features

# Define the features to remove
features_to_remove = {'encounter_id','patient_nbr'}

#Collect rest all features other than the above feature
approved_cols = [cols for cols in df.columns if (cols not in features_to_remove)]

# StatsOptions : which used to specify what and all the features are allowed to perform stats operations further.
stat_options = tfdv.StatsOptions(feature_allowlist = approved_cols)

# Review the features to generate the statistics
for feature in stat_options.feature_allowlist:
  print(feature)


race
gender
age
weight
admission_type_id
discharge_disposition_id
admission_source_id
time_in_hospital
payer_code
medical_specialty
num_lab_procedures
num_procedures
num_medications
number_outpatient
number_emergency
number_inpatient
diag_1
diag_2
diag_3
number_diagnoses
max_glu_serum
A1Cresult
metformin
repaglinide
nateglinide
chlorpropamide
glimepiride
acetohexamide
glipizide
glyburide
tolbutamide
pioglitazone
rosiglitazone
acarbose
miglitol
troglitazone
tolazamide
examide
citoglipton
insulin
glyburide-metformin
glipizide-metformin
glimepiride-pioglitazone
metformin-rosiglitazone
metformin-pioglitazone
change
diabetesMed
readmitted


#  Generating Training Statistics

In [ ]:
train_stats = tfdv.generate_statistics_from_dataframe(dataframe = train_df,
                                                      stats_options = stat_options)

# Number of features used to compute statistics
print(f"Number of features used : {len(train_stats.datasets[0].features)}")

# check the number of examples
print(f"number of examples used : {train_stats.datasets[0].num_examples}")

# Check the column names of the first column and last column feature
print(f"First Feature : {train_stats.datasets[0].features[0].path.step[0]}")
print(f"Last Feature : {train_stats.datasets[0].features[-1].path.step[0]}")

Number of features used : 48
number of examples used : 71236
First Feature : race
Last Feature : readmitted


# Visualize Training Statistics

In [ ]:
tfdv.visualize_statistics(train_stats)

# Infer the training set Schema

In [ ]:
# Infer the data schema by using training statistics
schema = tfdv.infer_schema(train_stats)

# Display the data schema
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'race',STRING,optional,single,'race'
'gender',STRING,required,,'gender'
'age',STRING,required,,'age'
'weight',STRING,optional,single,'weight'
'admission_type_id',INT,required,,-
'discharge_disposition_id',INT,required,,-
'admission_source_id',INT,required,,-
'time_in_hospital',INT,required,,-
'payer_code',STRING,optional,single,'payer_code'


,Values
Domain,
'race',"'AfricanAmerican', 'Asian', 'Caucasian', 'Hispanic', 'Other'"
'gender',"'Female', 'Male', 'Unknown/Invalid'"
'age',"'[0-10)', '[10-20)', '[20-30)', '[30-40)', '[40-50)', '[50-60)', '[60-70)', '[70-80)', '[80-90)', '[90-100)'"
'weight',"'>200', '[0-25)', '[100-125)', '[125-150)', '[150-175)', '[175-200)', '[25-50)', '[50-75)', '[75-100)'"
'payer_code',"'BC', 'CH', 'CM', 'CP', 'DM', 'HM', 'MC', 'MD', 'MP', 'OG', 'OT', 'PO', 'SI', 'SP', 'UN', 'WC'"
'medical_specialty',"'AllergyandImmunology', 'Anesthesiology', 'Anesthesiology-Pediatric', 'Cardiology', 'Cardiology-Pediatric', 'Dentistry', 'Dermatology', 'Emergency/Trauma', 'Endocrinology', 'Family/GeneralPractice', 'Gastroenterology', 'Gynecology', 'Hematology', 'Hematology/Oncology', 'Hospitalist', 'InfectiousDiseases', 'InternalMedicine', 'Nephrology', 'Neurology', 'Obsterics&Gynecology-GynecologicOnco', 'Obstetrics', 'ObstetricsandGynecology', 'Oncology', 'Ophthalmology', 'Orthopedics', 'Orthopedics-Reconstructive', 'Osteopath', 'Otolaryngology', 'OutreachServices', 'Pathology', 'Pediatrics', 'Pediatrics-AllergyandImmunology', 'Pediatrics-CriticalCare', 'Pediatrics-EmergencyMedicine', 'Pediatrics-Endocrinology', 'Pediatrics-Hematology-Oncology', 'Pediatrics-InfectiousDiseases', 'Pediatrics-Neurology', 'Pediatrics-Pulmonology', 'Perinatology', 'PhysicalMedicineandRehabilitation', 'PhysicianNotFound', 'Podiatry', 'Proctology', 'Psychiatry', 'Psychiatry-Addictive', 'Psychiatry-Child/Adolescent', 'Psychology', 'Pulmonology', 'Radiologist', 'Radiology', 'Rheumatology', 'Speech', 'SportsMedicine', 'Surgeon', 'Surgery-Cardiovascular', 'Surgery-Cardiovascular/Thoracic', 'Surgery-Colon&Rectal', 'Surgery-General', 'Surgery-Maxillofacial', 'Surgery-Neuro', 'Surgery-Pediatric', 'Surgery-Plastic', 'Surgery-PlasticwithinHeadandNeck', 'Surgery-Thoracic', 'Surgery-Vascular', 'SurgicalSpecialty', 'Urology'"
'max_glu_serum',"'>200', '>300', 'None', 'Norm'"
'A1Cresult',"'>7', '>8', 'None', 'Norm'"
'metformin',"'Down', 'No', 'Steady', 'Up'"


In [ ]:
# Check the number of features
print(f"Number of features : {len(schema.feature)}")
# checking domain name of 2nd feature
print(f"Second feature in schema : {list(schema.feature)[1].domain}")

Number of features : 48
Second feature in schema : gender


# Calculate , Visualize and Fix Evaluation Anomalies

## Compare Training and Evaluation Statistics



In [37]:
eval_stats = tfdv.generate_statistics_from_dataframe(dataframe = eval_df,
                                                     stats_options = stat_options)

tfdv.visualize_statistics(lhs_statistics = train_stats,
                          rhs_statistics = eval_stats,
                          lhs_name = "TRAINING",
                          rhs_name = "EVALUATION")


In [40]:
# Get the number of features used to compute statistics
print(f"Number of features : {len(eval_stats.datasets[0].features)}")

# Check the number of examples used
print(f"Number of Examples used : {eval_stats.datasets[0].num_examples}")

# Check the column names of first and last feature
print(f"First Feature : {eval_stats.datasets[0].features[0].path.step[0]}")
print(f"Last feature : {eval_stats.datasets[0].features[-1].path.step[0]}")

Number of features : 48
Number of Examples used : 15265
First Feature : race
Last feature : readmitted


# Detecting Anomalies
## In the "glimepiride-pioglitazone" has different ranges in training set and evaluation set .



In [45]:
train_df['glimepiride-pioglitazone'].describe()

count     71236
unique        1
top          No
freq      71236
Name: glimepiride-pioglitazone, dtype: object

In [46]:
eval_df['glimepiride-pioglitazone'].describe()

count     15265
unique        2
top          No
freq      15264
Name: glimepiride-pioglitazone, dtype: object

In [50]:
# Creating a function to detect and display anomalies in the dataset

def detect_and_display(statistics,schema):
  anomalies = tfdv.validate_statistics(statistics,schema)
  return tfdv.display_anomalies(anomalies)

detect_and_display(eval_stats,schema)

,Anomaly short description,Anomaly long description
Feature name,,
'medical_specialty',Unexpected string values,Examples contain values missing from the schema: Neurophysiology (<1%).
'glimepiride-pioglitazone',Unexpected string values,Examples contain values missing from the schema: Steady (<1%).


# Fix evaluation anomalies in the schema

## The values of features "medical_speciality" and "glimepride-pioglitazone" that were not included in the schema generated from the training data

In [56]:
# Get the domain associated with the input feature
glimpride_pioglitazone_domain = tfdv.get_domain(schema,'glimepiride-pioglitazone')
# print(glimpride_pioglitazone_domain)
# Append the missing vlaue "Steady" to the domain
glimpride_pioglitazone_domain.value.append("Steady")

# Get the domain assoicated with the input feature
medical_speciality_domain = tfdv.get_domain(schema,'medical_specialty')
# Append the missing value "Neurophysiology" to the domain
medical_speciality_domain.value.append("Neurophysiology")

detect_and_display(eval_stats,schema)

# Schema Environments

## check anomalies in the serving set

In [58]:
options = tfdv.StatsOptions(schema = schema,
                            infer_type_from_schema = True,
                            feature_allowlist = approved_cols)
options

<_generators=None, _feature_allowlist=['race', 'gender', 'age', 'weight', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'time_in_hospital', 'payer_code', 'medical_specialty', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'], _schema=feature {
  name: "race"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "race"
  presence {
    min_count: 1
  }
}
feature {
  name: "gender"
  typ

In [61]:
# Generate serving dataset statistics

serving_stats = tfdv.generate_statistics_from_dataframe(dataframe = serv_df,
                                         stats_options = stat_options)

# Calculate and display anomalies in the serving dataset
detect_and_display(serving_stats,schema)

,Anomaly short description,Anomaly long description
Feature name,,
'payer_code',Unexpected string values,Examples contain values missing from the schema: FR (<1%).
'metformin-rosiglitazone',Unexpected string values,Examples contain values missing from the schema: Steady (<1%).
'readmitted',Column dropped,Column is completely missing
'metformin-pioglitazone',Unexpected string values,Examples contain values missing from the schema: Steady (<1%).
'medical_specialty',Unexpected string values,"Examples contain values missing from the schema: DCPTEAM (<1%), Endocrinology-Metabolism (<1%), Resident (<1%)."


In [74]:
# fix the anomalies

# Get the feature and relax to match 90% of the domain
payer_code_domain = tfdv.get_feature(schema,"payer_code")
payer_code_domain.distribution_constraints.min_domain_mass = 0.9

# Get the feature and relax to match 90% of the domain
medical_speciality = tfdv.get_feature(schema,"medical_specialty")
medical_speciality.distribution_constraints.min_domain_mass = 0.9


# Detect again if any anomalies is present
detect_and_display(serving_stats,schema)

,Anomaly short description,Anomaly long description
Feature name,,
'readmitted',Column dropped,Column is completely missing
'metformin-pioglitazone',Unexpected string values,Examples contain values missing from the schema: Steady (<1%).


# Modifying the domain

In [75]:
tfdv.display_schema(schema)


,Type,Presence,Valency,Domain
Feature name,,,,
'race',STRING,optional,single,'race'
'gender',STRING,required,,'gender'
'age',STRING,required,,'age'
'weight',STRING,optional,single,'weight'
'admission_type_id',INT,required,,-
'discharge_disposition_id',INT,required,,-
'admission_source_id',INT,required,,-
'time_in_hospital',INT,required,,-
'payer_code',STRING,optional,single,'payer_code'


,Values
Domain,
'race',"'AfricanAmerican', 'Asian', 'Caucasian', 'Hispanic', 'Other'"
'gender',"'Female', 'Male', 'Unknown/Invalid'"
'age',"'[0-10)', '[10-20)', '[20-30)', '[30-40)', '[40-50)', '[50-60)', '[60-70)', '[70-80)', '[80-90)', '[90-100)'"
'weight',"'>200', '[0-25)', '[100-125)', '[125-150)', '[150-175)', '[175-200)', '[25-50)', '[50-75)', '[75-100)'"
'payer_code',"'BC', 'CH', 'CM', 'CP', 'DM', 'HM', 'MC', 'MD', 'MP', 'OG', 'OT', 'PO', 'SI', 'SP', 'UN', 'WC'"
'medical_specialty',"'AllergyandImmunology', 'Anesthesiology', 'Anesthesiology-Pediatric', 'Cardiology', 'Cardiology-Pediatric', 'Dentistry', 'Dermatology', 'Emergency/Trauma', 'Endocrinology', 'Family/GeneralPractice', 'Gastroenterology', 'Gynecology', 'Hematology', 'Hematology/Oncology', 'Hospitalist', 'InfectiousDiseases', 'InternalMedicine', 'Nephrology', 'Neurology', 'Obsterics&Gynecology-GynecologicOnco', 'Obstetrics', 'ObstetricsandGynecology', 'Oncology', 'Ophthalmology', 'Orthopedics', 'Orthopedics-Reconstructive', 'Osteopath', 'Otolaryngology', 'OutreachServices', 'Pathology', 'Pediatrics', 'Pediatrics-AllergyandImmunology', 'Pediatrics-CriticalCare', 'Pediatrics-EmergencyMedicine', 'Pediatrics-Endocrinology', 'Pediatrics-Hematology-Oncology', 'Pediatrics-InfectiousDiseases', 'Pediatrics-Neurology', 'Pediatrics-Pulmonology', 'Perinatology', 'PhysicalMedicineandRehabilitation', 'PhysicianNotFound', 'Podiatry', 'Proctology', 'Psychiatry', 'Psychiatry-Addictive', 'Psychiatry-Child/Adolescent', 'Psychology', 'Pulmonology', 'Radiologist', 'Radiology', 'Rheumatology', 'Speech', 'SportsMedicine', 'Surgeon', 'Surgery-Cardiovascular', 'Surgery-Cardiovascular/Thoracic', 'Surgery-Colon&Rectal', 'Surgery-General', 'Surgery-Maxillofacial', 'Surgery-Neuro', 'Surgery-Pediatric', 'Surgery-Plastic', 'Surgery-PlasticwithinHeadandNeck', 'Surgery-Thoracic', 'Surgery-Vascular', 'SurgicalSpecialty', 'Urology', 'Neurophysiology', 'Neurophysiology'"
'max_glu_serum',"'>200', '>300', 'None', 'Norm'"
'A1Cresult',"'>7', '>8', 'None', 'Norm'"
'metformin',"'Down', 'No', 'Steady', 'Up'"


# Many fearures have the same values . These values are common to many features including the ones with missing values during schema inference.
## To modify the domain of some features to match an existing domain.To address the detected anomaly,we can set the domain of these features.

In [101]:
# Creating a function to modify the features

def modify_domain_of_features(feature_list,schema,to_domain_name):
  for feature in feature_list:
    tfdv.set_domain(schema=schema,domain= to_domain_name,feature_path = feature)
    return schema

In [102]:
domain_change_features = ['repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
                          'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone',
                          'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide',
                          'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
                          'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone']

schema = modify_domain_of_features(domain_change_features, schema, 'metformin')

# Display the new schema
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'race',STRING,optional,single,'race'
'gender',STRING,required,,'gender'
'age',STRING,required,,'age'
'weight',STRING,optional,single,'weight'
'admission_type_id',INT,required,,-
'discharge_disposition_id',INT,required,,-
'admission_source_id',INT,required,,-
'time_in_hospital',INT,required,,-
'payer_code',STRING,optional,single,'payer_code'


,Values
Domain,
'race',"'AfricanAmerican', 'Asian', 'Caucasian', 'Hispanic', 'Other'"
'gender',"'Female', 'Male', 'Unknown/Invalid'"
'age',"'[0-10)', '[10-20)', '[20-30)', '[30-40)', '[40-50)', '[50-60)', '[60-70)', '[70-80)', '[80-90)', '[90-100)'"
'weight',"'>200', '[0-25)', '[100-125)', '[125-150)', '[150-175)', '[175-200)', '[25-50)', '[50-75)', '[75-100)'"
'payer_code',"'BC', 'CH', 'CM', 'CP', 'DM', 'HM', 'MC', 'MD', 'MP', 'OG', 'OT', 'PO', 'SI', 'SP', 'UN', 'WC'"
'medical_specialty',"'AllergyandImmunology', 'Anesthesiology', 'Anesthesiology-Pediatric', 'Cardiology', 'Cardiology-Pediatric', 'Dentistry', 'Dermatology', 'Emergency/Trauma', 'Endocrinology', 'Family/GeneralPractice', 'Gastroenterology', 'Gynecology', 'Hematology', 'Hematology/Oncology', 'Hospitalist', 'InfectiousDiseases', 'InternalMedicine', 'Nephrology', 'Neurology', 'Obsterics&Gynecology-GynecologicOnco', 'Obstetrics', 'ObstetricsandGynecology', 'Oncology', 'Ophthalmology', 'Orthopedics', 'Orthopedics-Reconstructive', 'Osteopath', 'Otolaryngology', 'OutreachServices', 'Pathology', 'Pediatrics', 'Pediatrics-AllergyandImmunology', 'Pediatrics-CriticalCare', 'Pediatrics-EmergencyMedicine', 'Pediatrics-Endocrinology', 'Pediatrics-Hematology-Oncology', 'Pediatrics-InfectiousDiseases', 'Pediatrics-Neurology', 'Pediatrics-Pulmonology', 'Perinatology', 'PhysicalMedicineandRehabilitation', 'PhysicianNotFound', 'Podiatry', 'Proctology', 'Psychiatry', 'Psychiatry-Addictive', 'Psychiatry-Child/Adolescent', 'Psychology', 'Pulmonology', 'Radiologist', 'Radiology', 'Rheumatology', 'Speech', 'SportsMedicine', 'Surgeon', 'Surgery-Cardiovascular', 'Surgery-Cardiovascular/Thoracic', 'Surgery-Colon&Rectal', 'Surgery-General', 'Surgery-Maxillofacial', 'Surgery-Neuro', 'Surgery-Pediatric', 'Surgery-Plastic', 'Surgery-PlasticwithinHeadandNeck', 'Surgery-Thoracic', 'Surgery-Vascular', 'SurgicalSpecialty', 'Urology', 'Neurophysiology', 'Neurophysiology'"
'max_glu_serum',"'>200', '>300', 'None', 'Norm'"
'A1Cresult',"'>7', '>8', 'None', 'Norm'"
'metformin',"'Down', 'No', 'Steady', 'Up'"


In [104]:
print(f"Domain name of 'chlorpropamide': {tfdv.get_feature(schema, 'chlorpropamide').domain}")
print(f"Domain values of 'chlorpropamide': {tfdv.get_domain(schema, 'chlorpropamide').value}")

print(f"Domain name of 'repaglinide': {tfdv.get_feature(schema, 'repaglinide').domain}")
print(f"Domain values of 'repaglinide': {tfdv.get_domain(schema, 'repaglinide').value}")

print(f"Domain name of 'nateglinide': {tfdv.get_feature(schema, 'nateglinide').domain}")
print(f"Domain values of 'nateglinide': {tfdv.get_domain(schema, 'nateglinide').value}")

Domain name of 'chlorpropamide': metformin
Domain values of 'chlorpropamide': ['Down', 'No', 'Steady', 'Up']
Domain name of 'repaglinide': metformin
Domain values of 'repaglinide': ['Down', 'No', 'Steady', 'Up']
Domain name of 'nateglinide': metformin
Domain values of 'nateglinide': ['Down', 'No', 'Steady', 'Up']


In [105]:
# Checking the anomalies again

detect_and_display(serving_stats,schema)

,Anomaly short description,Anomaly long description
Feature name,,
'readmitted',Column dropped,Column is completely missing


# Detecting anomalies with Environment

## Still the readmitted feature is been detected as anomaly which we have dropped early. let's ignore this detected anomaly in the serving data

In [106]:
# all features are by default in both training and serving environment

schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

In [109]:
# Specify the "readmitted" feature is not in serving environment
tfdv.get_feature(schema,'readmitted').not_in_environment.append("SERVING")

# Inferred schema and serving environment parameter
serving_anomalies_with_env = tfdv.validate_statistics(serving_stats,schema,environment = "SERVING")

In [111]:
# Display anomalies
tfdv.display_anomalies(serving_anomalies_with_env)

# Check for data Drift and Skew

### Data Skew : Uneven distribution of the values across the feature.

### Data drift : Data changes over the time as the world changes.

In [120]:
## DATA SKEW Example.

# Calculate skew for the diabetesMed feature
diabeted_med = tfdv.get_feature(schema,"diabetesMed")
# If it exceeds the threshold , it will shown as anomalies
diabeted_med.skew_comparator.infinity_norm.threshold = 0.035

In [121]:
## DATA DRIFT Example

# calculate drift for the payer_code feature
payer_code = tfdv.get_feature(schema,'payer_code')
# If it exceeds the threshold value , then it is detected as anomalies
payer_code.drift_comparator.infinity_norm.threshold = 0.035

In [122]:
# Calculate anomalies
skew_drift_anomalies = tfdv.validate_statistics(train_stats,
                                                schema,
                                                previous_statistics = eval_stats,
                                                serving_statistics = serving_stats)
# Display anomalies

tfdv.display_anomalies(skew_drift_anomalies)